In [1]:
import os
import librosa
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
%matplotlib inline


# Step 2: Prepare the Audio Data
## 2.1: Define Audio Preprocessing Functions

In [2]:
def normalize_loudness(y):
    """Normalize audio to have consistent loudness."""
    rms = np.sqrt(np.mean(y**2))
    if rms > 0:
        y = y / rms
    return y

def load_and_preprocess_audio(file_path, target_sr=16000):
    """Load and preprocess audio file."""
    y, sr = librosa.load(file_path, sr=None)
    # Downsample if necessary
    if sr != target_sr:
        y = librosa.resample(y, orig_sr=sr, target_sr=target_sr)
    # Normalize loudness
    y = normalize_loudness(y)
    return y


# 2.2: Load Audio Files

In [3]:
# Path to your dataset folder
dataset_path = 'dataset'

# Get list of audio files
audio_files = [os.path.join(dataset_path, f) for f in os.listdir(dataset_path) if f.endswith('.wav')]

# Preprocess and store audio data
audio_data = []

for file in tqdm(audio_files, desc='Loading and preprocessing audio files'):
    y = load_and_preprocess_audio(file)
    audio_data.append(y)

Loading and preprocessing audio files:   0%|          | 0/615 [00:00<?, ?it/s]

Step 3: Extract Features for RNN Input
3.1: Define Feature Extraction Function

In [4]:
def extract_mfcc_features(y, sr=16000, n_mfcc=40):
    """Extract MFCC features from audio signal."""
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfcc = mfcc.T  # Shape: (Time steps, n_mfcc)
    return mfcc

3.2: Extract Features from All Audio Data

In [5]:
# List to hold feature sequences
feature_sequences = []

for y in tqdm(audio_data, desc='Extracting features'):
    features = extract_mfcc_features(y)
    feature_sequences.append(features)


Extracting features:   0%|          | 0/615 [00:00<?, ?it/s]

Step 4: Prepare Data for PyTorch
4.1: Create a Custom Dataset

In [6]:
class AudioDataset(Dataset):
    def __init__(self, feature_sequences):
        self.feature_sequences = feature_sequences

    def __len__(self):
        return len(self.feature_sequences)

    def __getitem__(self, idx):
        return self.feature_sequences[idx]
    

4.2: Define a Collate Function to Handle Variable Sequence Lengths

In [7]:
def collate_fn(batch):
    # Batch is a list of sequences
    # Find the longest sequence in the batch
    batch_size = len(batch)
    lengths = [seq.shape[0] for seq in batch]
    max_length = max(lengths)

    # Number of features (n_mfcc)
    n_features = batch[0].shape[1]

    # Create a tensor of zeros with shape (batch_size, max_length, n_features)
    padded_batch = torch.zeros(batch_size, max_length, n_features)

    for i, seq in enumerate(batch):
        length = seq.shape[0]
        padded_batch[i, :length, :] = torch.tensor(seq)

    return padded_batch, lengths

4.3: Create DataLoader

In [8]:
dataset = AudioDataset(feature_sequences)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)

Step 5: Define the RNN Autoencoder Model
5.1: Define the Model

In [9]:
class RNNAutoencoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(RNNAutoencoder, self).__init__()
        self.hidden_size = hidden_size

        # Encoder
        self.encoder_rnn = nn.GRU(input_size, hidden_size, batch_first=True)

        # Decoder
        self.decoder_rnn = nn.GRU(hidden_size, hidden_size, batch_first=True)

        # Output layer to map hidden_size back to input_size
        self.output_layer = nn.Linear(hidden_size, input_size)

    def forward(self, x, lengths):
        # Pack the sequence
        packed_input = nn.utils.rnn.pack_padded_sequence(
            x, lengths, batch_first=True, enforce_sorted=False
        )

        # Encoder
        _, hidden = self.encoder_rnn(packed_input)

        # Decoder input: zeros
        batch_size, max_length, _ = x.size()
        decoder_input = torch.zeros(batch_size, max_length, self.hidden_size).to(x.device)

        # Decoder hidden state
        decoder_hidden = hidden  # Now sizes match

        # Decoder
        outputs, _ = self.decoder_rnn(decoder_input, decoder_hidden)

        # Map outputs to input_size
        outputs = self.output_layer(outputs)

        return outputs

5.2: Initialize the Model

In [10]:
input_size = feature_sequences[0].shape[1]  # n_mfcc
hidden_size = 128  # Adjust as needed

model = RNNAutoencoder(input_size, hidden_size)


Step 6: Define Loss Function and Optimizer

In [11]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Step 7: Train the Model

In [12]:
# Training loop
num_epochs = 20  # Adjust as needed
device = torch.device( 'cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    for batch_data, lengths in tqdm(dataloader, desc=f'Epoch {epoch+1}/{num_epochs}'):
        # Move data to device
        batch_data = batch_data.to(device).float()

        # Forward pass
        outputs = model(batch_data, lengths)

        # Compute loss
        loss = criterion(outputs, batch_data)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')


Epoch 1/20:   0%|          | 0/39 [00:00<?, ?it/s]

Epoch [1/20], Loss: 374.3048


Epoch 2/20:   0%|          | 0/39 [00:00<?, ?it/s]

Epoch [2/20], Loss: 356.4214


Epoch 3/20:   0%|          | 0/39 [00:00<?, ?it/s]

...

Epoch [18/20], Loss: 322.3578


Epoch 19/20:   0%|          | 0/39 [00:00<?, ?it/s]

Epoch [19/20], Loss: 322.3107


Epoch 20/20:   0%|          | 0/39 [00:00<?, ?it/s]

Epoch [20/20], Loss: 316.7708
